# MIST101 Pratical 5: Mixture of Bernoulli

## 1. Intro: 
In this tutorial you'll learn how to train an unsupervised model with maximium likelihood estimation (MLE) / maximum a posteriori (MAP) on MNIST. The model we'll use is a mixture of Bernoulli. The word **mixture** comes from the fact that we have a latent variable $z$ that could take discrete integer values ranging from 1 to 10. Our hope is that eventually, our model could learn a conditional distribution $p(x|z, \theta)$, s.t. given the specific value of $z$ it could be able to present us a sensible probability mass function which is a product of Bernoullis that match that of any digit (1 to 10) we are modeling.

## 2. Recap of model definition: 

- $p(z)$ is defined to be a uniform categorical random variable which could take values 1,2,...10

- $p( x_d | z, \theta_d)$ is defined to be a Bernoulli distribution ($x_d$ is the $d$-th pixel):

$$p( x_d | z, \theta_d) = {\theta_d}^{x_d}  (1 - \theta_d)^{(1 - x_d)} $$

- for numerical stability, we could write everything is log-space:

$$\log p( x_d | z, \theta_d) = x_d \log \theta_d + (1 - x_d)\log (1 - \theta_d) $$

- if we perform MLE, the objective we want to maximize is ($x^{(i)}$ denotes the $i$-th data point):

$$\sum_{i} \log  \sum_{z^{(i)}} p(z^{(i)}) \prod_{d} p(x_d^{(i)} | z^{(i)}, \theta) \tag{1} $$

- for numerical stability, we could use the numpy/tensorflow builtin stable version of **logsumexp**:

$$\text{logsumexp}(v_1, v_2, ..., v_n) = \log \sum_{i} \exp(v_i)$$

- this means when we want to compute a logsum, we could first take the log of each component and then pass to logsumexp:

$$\text{logsum}(u_1, u_2, ..., u_n) = \text{logsumexp}(\log u_1, \log u_2, ..., \log u_n)$$

- our loss function (1) could be rewritten as:

$$\sum_{i} \text{logsumexp} (\text{logit}_1, \text{logit}_2, ..., \text{logit}_n ) \tag{2} $$

$$\text{logit}_j = \log p(z^{(i)}=j) \prod_{d} p(x_d^{(i)} | z^{(i)}=j, \theta) = \log \frac{1}{10} + \sum_d \log p(x_d^{(i)} | z^{(i)} = j , \theta)$$

In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import matplotlib
import numpy as np

mnist = input_data.read_data_sets('MNIST_data/')
train = mnist.train.images
test  = mnist.test.images

# in place binarization without copy
train[train > 0.5] = 1. ; train[train <= 0.5] = 0.
test[test > 0.5] = 1. ; test[test <= 0.5] = 0.

In [ ]:
# feel free to tweak the hyperparameters
lr = 1e-4                                      # learning rate for optimizer
categories = 10                                # number of categories for latent z
epochs = 10                                    # number of epochs for training
batch_size = 128                               # batch size
train_iters = train.shape[0] // batch_size     # number of iterations each epoch
log_every = 1                                  # print training status every such epochs

In [ ]:
x = tf.placeholder(dtype=tf.float32, shape=[None, 784])
theta = tf.get_variable(name='theta', shape=[categories, 784])

# TODO: define loss function here according to equation (2) above
# Hint: you might find tf.reduce_logsumexp useful
loss = tf.reduce_sum(theta)  # this is just a stub, you should modify this

train_op = tf.train.AdamOptimizer(lr).minimize(-loss)
init_op = tf.global_variables_initializer()

In [ ]:
with tf.Session() as sess:
    sess.run(init_op)
    
    for e in range(epochs):
        for i in range(train_iters):
            batch = train[i*batch_size:(i+1)*batch_size]
            sess.run(train_op, feed_dict={x: batch})
        
        if e % log_every == 0:
            stats = sess.run(-loss, feed_dict={x: test})
            print ('aggregate marginal log-likelihood on test set %.4f' % stats)